In [1]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 10:08:32] [setup] RAM Tracking...
[codecarbon INFO @ 10:08:32] [setup] CPU Tracking...
[codecarbon WARNING @ 10:08:35] No CPU tracking mode found. Falling back on CPU constant mode. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 10:08:37] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 10:08:37] [setup] GPU Tracking...
[codecarbon INFO @ 10:08:37] No GPU found.
[codecarbon INFO @ 10:08:37] >>> Tracker's metadata:
[codecarbon INFO @ 10:08:37]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 10:08:37]   Python version: 3.9.12
[codecarbon INFO @ 10:08:37]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:08:37]   Available RAM : 8.000 GB
[codecarbon INFO @ 10:08:37]   CPU count: 8
[codecarbon INFO @ 10:08:37]   CPU model: Apple M1
[codecarbon INFO @ 10:08:37]   GPU count: None
[codecarbon INFO @ 10:08:37]   GPU model: None
[codecarbon INFO @ 10:08:40] Saving emissions data to

In [ ]:
import pandas as pd

# Carga del CSV
df = pd.read_csv('../training.csv')  # ../ sube un nivel desde /SVM/

# Primer vistazo
print(df.shape)
df.head()


(250000, 33)


,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [3]:
import numpy as np

# Codificar la variable objetivo: 's' → 1, 'b' → 0
df['Label'] = df['Label'].map({'s': 1, 'b': 0})

# Reemplazar -999.0 por NaN
df.replace(-999.0, np.nan, inplace=True)


In [4]:
# Eliminar columnas que no usaremos
df.drop(['EventId', 'Weight'], axis=1, inplace=True)

# Separar X (features) e y (target)
X = df.drop('Label', axis=1)
y = df['Label']


In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42, stratify=y
)


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Reducimos el conjunto de entrenamiento al 75% por tuning.

In [8]:
from sklearn.model_selection import train_test_split

# Subconjunto del 50% para tuning
X_sub, _, y_sub, _ = train_test_split(
    X_train_scaled, y_train, train_size=0.75, stratify=y_train, random_state=42
)

print(f"Subconjunto para tuning: {X_sub.shape[0]} muestras")


Subconjunto para tuning: 150000 muestras


In [9]:
import time
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Lista de configuraciones a probar
# Nuevas combinaciones más variadas
svm_configuraciones = [
    {'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}, 
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1},
    {'kernel': 'poly', 'C': 1.0, 'gamma': 'scale'},
    {'kernel': 'poly', 'C': 0.5, 'gamma': 'scale'},
    {'kernel': 'linear', 'C': 1.0},
    {'kernel': 'sigmoid', 'C': 5.0, 'gamma': 0.01}
]


# Guardar resultados
resultados_sub = {}

for i, config in enumerate(svm_configuraciones):
    print(f"\n⏳ Entrenando SVM #{i+1}: {config}")
    inicio = time.time()
    
    modelo = SVC(**config, random_state=42)
    modelo.fit(X_sub, y_sub)
    y_pred = modelo.predict(X_test_scaled)  # ¡Seguimos evaluando en el test completo!
    
    fin = time.time()
    
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    resultados_sub[f"SVM_{i+1}_{config}"] = {
        'accuracy': acc,
        'recall_1': report['1']['recall'],
        'f1_1': report['1']['f1-score'],
        'tiempo': fin - inicio
    }

    print(f"Terminado en {fin - inicio:.2f} s - Accuracy: {acc:.4f}")


⏳ Entrenando SVM #1: {'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'}


[codecarbon INFO @ 10:08:55] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:08:55] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:08:55] 0.000033 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:09:10] Energy consumed for RAM : 0.000025 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:09:10] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:09:10] 0.000067 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:09:25] Energy consumed for RAM : 0.000038 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:09:25] Energy consumed for all CPUs : 0.000063 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:09:25] 0.000100 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:09:40] Energy consumed for RAM : 0.000050 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:09:40] Energy consumed for all CPUs : 0.000083 kWh. Total CPU Power : 5.0 W
[codecarbon

Terminado en 973.33 s - Accuracy: 0.8303

⏳ Entrenando SVM #2: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}


[codecarbon INFO @ 10:25:10] Energy consumed for RAM : 0.000825 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:25:10] Energy consumed for all CPUs : 0.001376 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:25:10] 0.002201 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:25:25] Energy consumed for RAM : 0.000838 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:25:25] Energy consumed for all CPUs : 0.001396 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:25:25] 0.002234 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:25:40] Energy consumed for RAM : 0.000850 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:25:40] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 10:25:40] 0.002267 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:25:55] Energy consumed for RAM : 0.000863 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 10:25:55] Energy consumed for all CPUs : 0.001438 kWh. Total CPU Power : 5.0 W
[codecarbon

Terminado en 1015.56 s - Accuracy: 0.8320

⏳ Entrenando SVM #3: {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1}


...

Terminado en 1007.91 s - Accuracy: 0.8077

⏳ Entrenando SVM #6: {'kernel': 'linear', 'C': 1.0}


[codecarbon INFO @ 11:50:28] Energy consumed for RAM : 0.005088 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:50:28] Energy consumed for all CPUs : 0.008482 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:50:28] 0.013570 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:50:43] Energy consumed for RAM : 0.005101 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:50:43] Energy consumed for all CPUs : 0.008503 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:50:43] 0.013604 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:50:43] 0.000387 g.CO2eq/s mean an estimation of 12.19586359439669 kg.CO2eq/year
[codecarbon INFO @ 11:50:58] Energy consumed for RAM : 0.005113 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:50:58] Energy consumed for all CPUs : 0.008524 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 11:50:58] 0.013637 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:51:13] Energy consumed for RAM : 0.005126 kWh. RAM Power : 3.0 W
[codeca

Terminado en 3113.96 s - Accuracy: 0.7492

⏳ Entrenando SVM #7: {'kernel': 'sigmoid', 'C': 5.0, 'gamma': 0.01}


[codecarbon INFO @ 12:42:29] Energy consumed for RAM : 0.007688 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:42:29] Energy consumed for all CPUs : 0.012817 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:42:29] 0.020505 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:42:44] Energy consumed for RAM : 0.007701 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:42:44] Energy consumed for all CPUs : 0.012838 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:42:44] 0.020539 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:42:44] 0.000387 g.CO2eq/s mean an estimation of 12.195784810848327 kg.CO2eq/year
[codecarbon INFO @ 12:42:59] Energy consumed for RAM : 0.007713 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:42:59] Energy consumed for all CPUs : 0.012859 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:42:59] 0.020572 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:43:14] Energy consumed for RAM : 0.007726 kWh. RAM Power : 3.0 W
[codec

Terminado en 518.67 s - Accuracy: 0.6796


In [10]:
print("\nResumen tuning con 75% de datos:")
for nombre, res in resultados_sub.items():
    print(f"\nModelo: {nombre}")
    print(f"Accuracy: {res['accuracy']:.4f}")
    print(f"Recall clase 1: {res['recall_1']:.4f}")
    print(f"F1-score clase 1: {res['f1_1']:.4f}")
    print(f"Tiempo: {res['tiempo']:.2f} s")



Resumen tuning con 75% de datos:

Modelo: SVM_1_{'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'}
Accuracy: 0.8303
Recall clase 1: 0.6957
F1-score clase 1: 0.7376
Tiempo: 973.33 s

Modelo: SVM_2_{'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}
Accuracy: 0.8320
Recall clase 1: 0.7004
F1-score clase 1: 0.7408
Tiempo: 1015.56 s

Modelo: SVM_3_{'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1}
Accuracy: 0.8280
Recall clase 1: 0.6828
F1-score clase 1: 0.7312
Tiempo: 1646.31 s

Modelo: SVM_4_{'kernel': 'poly', 'C': 1.0, 'gamma': 'scale'}
Accuracy: 0.8113
Recall clase 1: 0.6260
F1-score clase 1: 0.6945
Tiempo: 1459.29 s

Modelo: SVM_5_{'kernel': 'poly', 'C': 0.5, 'gamma': 'scale'}
Accuracy: 0.8077
Recall clase 1: 0.6064
F1-score clase 1: 0.6836
Tiempo: 1007.91 s

Modelo: SVM_6_{'kernel': 'linear', 'C': 1.0}
Accuracy: 0.7492
Recall clase 1: 0.5401
F1-score clase 1: 0.5961
Tiempo: 3113.96 s

Modelo: SVM_7_{'kernel': 'sigmoid', 'C': 5.0, 'gamma': 0.01}
Accuracy: 0.6796
Recall clase 1: 0.5287
F1-score clase 1: 

Entrenar el modelo final con el 100% de X_train_scaled

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

print("\nEntrenando el modelo final con todo el conjunto de entrenamiento...")

inicio = time.time()

final_svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
final_svm.fit(X_train_scaled, y_train)

fin = time.time()


Entrenando el modelo final con todo el conjunto de entrenamiento...


[codecarbon INFO @ 12:50:59] Energy consumed for RAM : 0.008113 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:50:59] Energy consumed for all CPUs : 0.013526 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:50:59] 0.021639 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:51:14] Energy consumed for RAM : 0.008126 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:51:14] Energy consumed for all CPUs : 0.013546 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:51:14] 0.021672 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:51:29] Energy consumed for RAM : 0.008138 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:51:29] Energy consumed for all CPUs : 0.013567 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:51:29] 0.021706 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:51:44] Energy consumed for RAM : 0.008151 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 12:51:44] Energy consumed for all CPUs : 0.013588 kWh. Total CPU Power : 5.0 W
[codecarbon

In [12]:
# Predicción en test
y_pred_final = final_svm.predict(X_test_scaled)

# Métricas
acc_final = accuracy_score(y_test, y_pred_final)
report_final = classification_report(y_test, y_pred_final, output_dict=True)
conf_final = confusion_matrix(y_test, y_pred_final)

# Mostrar resultados
print("\n Modelo final entrenado en {:.2f} segundos".format(fin - inicio))
print("\n Classification Report (clase 1):")
print("Accuracy:", round(acc_final, 4))
print("Recall clase 1:", round(report_final['1']['recall'], 4))
print("F1-score clase 1:", round(report_final['1']['f1-score'], 4))
print("\n Matriz de Confusión:")
print(conf_final)


[codecarbon INFO @ 13:10:30] Energy consumed for RAM : 0.009088 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:10:30] Energy consumed for all CPUs : 0.015151 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:10:30] 0.024240 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:10:45] Energy consumed for RAM : 0.009101 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:10:45] Energy consumed for all CPUs : 0.015172 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:10:45] 0.024273 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:10:45] 0.000387 g.CO2eq/s mean an estimation of 12.19528019898859 kg.CO2eq/year
[codecarbon INFO @ 13:11:00] Energy consumed for RAM : 0.009113 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:11:00] Energy consumed for all CPUs : 0.015193 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:11:00] 0.024306 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:11:15] Energy consumed for RAM : 0.009126 kWh. RAM Power : 3.0 W
[codeca


 Modelo final entrenado en 1171.00 segundos

 Classification Report (clase 1):
Accuracy: 0.8338
Recall clase 1: 0.7036
F1-score clase 1: 0.7437

 Matriz de Confusión:
[[29635  3232]
 [ 5078 12055]]


In [13]:
from sklearn.metrics import roc_auc_score

# Probabilidades para clase 1
y_proba_final = final_svm.decision_function(X_test_scaled)

# Calcular AUC-ROC
auc_final = roc_auc_score(y_test, y_proba_final)
print("\n AUC-ROC:", round(auc_final, 4))


[codecarbon INFO @ 13:17:45] Energy consumed for RAM : 0.009451 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:17:45] Energy consumed for all CPUs : 0.015756 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:17:45] 0.025207 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:18:00] Energy consumed for RAM : 0.009464 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:18:00] Energy consumed for all CPUs : 0.015776 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:18:00] 0.025240 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:18:15] Energy consumed for RAM : 0.009476 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:18:15] Energy consumed for all CPUs : 0.015797 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:18:15] 0.025273 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:18:30] Energy consumed for RAM : 0.009489 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:18:30] Energy consumed for all CPUs : 0.015818 kWh. Total CPU Power : 5.0 W
[codecarbon


 AUC-ROC: 0.8954


In [14]:
from codecarbon import EmissionsTracker

emissions = tracker.stop()

[codecarbon INFO @ 13:24:35] Energy consumed for RAM : 0.009793 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:24:35] Energy consumed for all CPUs : 0.016325 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:24:35] 0.026118 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:24:35] 0.000387 g.CO2eq/s mean an estimation of 12.19566256127647 kg.CO2eq/year
